In [105]:
import json
import pandas as pd
import warnings
import seaborn as sns
from pyprojroot import here
import plotly.express as px

methods = ["JustCopy", "TimeGAN", "Time-Transformer", "TransFusion", "TTS-GAN", "TimeVQVAE"]
datasets = ["D2", "D3", "D4", "D5", "D6", "D7"]

rows = []

for method in methods:
    for dataset in datasets:
        file_candidates = list(here('result').glob(f'numeric_{method}_{dataset}_*.json'))
        if len(file_candidates) != 1:
            warnings.warn(f"Ignoring {method} {dataset}: Expected one result file for {method} {dataset}, instead matched {file_candidates}.")
            continue
        file_path = file_candidates[0]
        
        with open(file_path) as f:
            data = json.load(f)
        
        rows.append(((method, dataset), data))

df_all = pd.DataFrame.from_dict(dict(rows), orient="index")
df_all.index = pd.MultiIndex.from_tuples(df_all.index, names=["Method", "Dataset"])

# load timings
def timings_path(method):
    paths = list(here("models").glob(f"*{method}*/timings.csv"))
    assert len(paths) == 1
    return paths[0]

timings = pd.DataFrame()
for method in methods:
    df = pd.read_csv(timings_path(method))
    df["Method"] = method
    timings = pd.concat([timings, df], ignore_index=True)

_timings = timings.melt(id_vars="Method", var_name="Dataset", value_name="Time").set_index(["Method", "Dataset"]).sort_index(level="Method")

df_all = pd.concat([df_all, _timings], axis = 1)

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning:

Ignoring JustCopy D6: Expected one result file for JustCopy D6, instead matched [].

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:17: UserWarning:

Ignoring JustCopy D7: Expected one result file for JustCopy D7, instead matched [].



In [106]:
cm = sns.diverging_palette(h_neg=130, h_pos=0, as_cmap=True)

def style_results_table(df):
    return df.style\
    .background_gradient(cmap=cm, vmin=0, vmax=0.5, subset=["DS", "PS"])\
    .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["C-FID"])\
    .background_gradient(cmap=cm, vmin=0, vmax=2, subset=["MDD"])\
    .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["ACD"])\
    .background_gradient(cmap=cm, vmin=0, vmax=1, subset=["SD"])\
    .background_gradient(cmap=cm, vmin=0, vmax=3, subset=["KD"])\
    .background_gradient(cmap=cm, vmin=0, vmax=3, subset=["ED"])\
    .background_gradient(cmap=cm, vmin=0, vmax=15, subset=["DTW"])\
    .background_gradient(cmap=cm, vmin=0, vmax=1000, subset=["Time"])\
    .format({"DTW": "{:.2e}", "C-FID": "{:.2e}", "ED": "{:.2e}"})


style_results_table(df_all)

In [107]:
style_results_table(df_all.swaplevel("Method", "Dataset")\
    .sort_index(level=["Dataset", "Method"]))

## Spider Plots

In [125]:
rankings = df_all.unstack(level=1).rank()

ranking_by_metric = rankings.stack().reset_index().groupby("Method").mean()
spider_by_metric = ranking_by_metric.reset_index().melt(id_vars="Method")
px.line_polar(spider_by_metric, r="value", theta="variable", color="Method", line_close=True) \
    .update_layout(polar={"radialaxis": {"range": [6.9, 1], "dtick": 1}})

In [124]:
ranking_by_dataset = rankings.stack(level=0).reset_index().groupby("Method").mean()
spider_by_dataset = ranking_by_dataset.reset_index().melt(id_vars="Method")
px.line_polar(spider_by_dataset, r="value", theta="Dataset", color="Method", line_close=True) \
    .update_layout(polar={"radialaxis": {"range": [6.9,1], "dtick": 1}})

In [110]:
import plotly.graph_objects as go


def plot_average_rankings(df, title):
    s = df.unstack(level=1).rank().mean(axis=1)

    # Alternate y-offsets to avoid overlap
    y_offsets = [1, -1, 1, 1, 1]

    fig = go.Figure(go.Scatter(
        x=s.values,
        y=[0]*len(s),
        mode="markers+text",
        text=s.index,
        textposition=["top center" if y>0 else "bottom center" for y in y_offsets],
        marker=dict(size=12, color="steelblue")
    ))

    range_max = len(methods)

    # Thin horizontal number line
    fig.add_shape(type="line",
                x0=1, x1=range_max,
                y0=0, y1=0,
                line=dict(color="black", width=1))

    # Layout tweaks for minimal look
    fig.update_yaxes(visible=False)
    fig.update_xaxes(range=[0.8, range_max +0.2], showgrid=True, zeroline=False)
    fig.update_layout(
        height=200,
        xaxis_title="Average Rank",
        yaxis_title="",
        showlegend=False,
        margin=dict(t=60, b=60, l=150, r=150),
        title=title,
    )

    fig.show()

plot_average_rankings(df_all, "All Metrics")


In [111]:
import scikit_posthocs as sp

def conover_test(df):
    df = df.unstack(level=1).rank().T.reset_index().melt(id_vars=['level_0', 'Dataset'], var_name='Method', value_name='Rank')
    posthoc = sp.posthoc_conover(df, group_col='Method', val_col='Rank', p_adjust='holm')

    def highlight_below_threshold(val):
        color = 'background-color: turquoise' if val < 0.01 else ''
        return color

    return posthoc.style.applymap(highlight_below_threshold).format("{:.2e}")

conover_test(df_all)

,JustCopy,TTS-GAN,Time-Transformer,TimeGAN,TimeVQVAE,TransFusion
JustCopy,1.00e+00,2.26e-59,3.84e-25,4.53e-28,2.56e-20,4.85e-11
TTS-GAN,2.26e-59,1.00e+00,3.13e-18,1.91e-15,2.10e-23,5.96e-36
Time-Transformer,3.84e-25,3.13e-18,1.00e+00,3.80e-01,2.59e-01,9.24e-06
TimeGAN,4.53e-28,1.91e-15,3.80e-01,1.00e+00,5.10e-02,1.30e-07
TimeVQVAE,2.56e-20,2.10e-23,2.59e-01,5.10e-02,1.00e+00,3.82e-03
TransFusion,4.85e-11,5.96e-36,9.24e-06,1.30e-07,3.82e-03,1.00e+00


In [112]:
plot_average_rankings(df_all.drop(columns=["PS", "Time"]), "Fidelity")

In [113]:
conover_test(df_all.drop(columns=["PS", "Time"]))

,JustCopy,TTS-GAN,Time-Transformer,TimeGAN,TimeVQVAE,TransFusion
JustCopy,1.00e+00,9.32e-53,2.79e-24,9.65e-23,4.39e-17,1.06e-08
TTS-GAN,9.32e-53,1.00e+00,8.14e-16,2.34e-17,9.34e-24,1.15e-35
Time-Transformer,2.79e-24,8.14e-16,1.00e+00,5.97e-01,3.87e-02,6.80e-08
TimeGAN,9.65e-23,2.34e-17,5.97e-01,1.00e+00,9.90e-02,8.91e-07
TimeVQVAE,4.39e-17,9.34e-24,3.87e-02,9.90e-02,1.00e+00,3.29e-03
TransFusion,1.06e-08,1.15e-35,6.80e-08,8.91e-07,3.29e-03,1.00e+00


In [114]:
plot_average_rankings(df_all.drop(columns=["Time"]), "Without Timing")

In [115]:
conover_test(df_all.drop(columns=["Time"]))

,JustCopy,TTS-GAN,Time-Transformer,TimeGAN,TimeVQVAE,TransFusion
JustCopy,1.00e+00,8.47e-58,2.64e-27,3.28e-24,5.56e-20,1.26e-08
TTS-GAN,8.47e-58,1.00e+00,2.89e-16,2.29e-19,4.54e-24,2.36e-40
Time-Transformer,2.64e-27,2.89e-16,1.00e+00,3.30e-01,5.48e-02,4.41e-10
TimeGAN,3.28e-24,2.29e-19,3.30e-01,1.00e+00,3.27e-01,8.33e-08
TimeVQVAE,5.56e-20,4.54e-24,5.48e-02,3.27e-01,1.00e+00,6.04e-05
TransFusion,1.26e-08,2.36e-40,4.41e-10,8.33e-08,6.04e-05,1.00e+00


In [116]:
plot_average_rankings(df_all["PS"], "Predictive Score")

In [117]:
conover_test(df_all["PS"].to_frame())

,JustCopy,TTS-GAN,Time-Transformer,TimeGAN,TimeVQVAE,TransFusion
JustCopy,1.00e+00,8.22e-05,5.62e-03,7.88e-02,5.79e-03,8.16e-01
TTS-GAN,8.22e-05,1.00e+00,5.05e-01,3.95e-02,5.05e-01,1.73e-04
Time-Transformer,5.62e-03,5.05e-01,1.00e+00,7.61e-01,9.65e-01,1.70e-02
TimeGAN,7.88e-02,3.95e-02,7.61e-01,1.00e+00,7.61e-01,2.82e-01
TimeVQVAE,5.79e-03,5.05e-01,9.65e-01,7.61e-01,1.00e+00,1.74e-02
TransFusion,8.16e-01,1.73e-04,1.70e-02,2.82e-01,1.74e-02,1.00e+00


In [118]:
from scipy.stats import friedmanchisquare

df = df_all.unstack(level=1).T
ranks = df.rank(axis=0, method='average', ascending=False)

stat, p = friedmanchisquare(*[ranks.loc[m].values for m in ranks.index])
p

1.3455204629986705e-12

In [129]:
import pandas as pd

def leaderboard(df):
    # Create a new DataFrame with the same columns
    out = pd.DataFrame(index=df.index.levels[1], columns=df.columns)

    for lvl2 in df.index.levels[1]:
        # Select all rows with this level2
        subset = df.xs(lvl2, level=1)
        # Find max in each column
        min_idx = subset.idxmin()  # gives Level1 with max for each column
        out.loc[lvl2] = min_idx


    def color_cells(value):
        if value == "TTS-GAN":
            return 'background-color: yellow'
        if value == "TransFusion":
            return 'background-color: lightgreen'
        if value == "TimeGAN":
            return 'background-color: lightblue'
        if value == "Time-Transformer":
            return 'background-color: pink'
        if value == "TimeVQVAE":
            return 'background-color: turquoise'

    return out.style.applymap(color_cells)

leaderboard(df_all.drop(index="JustCopy", level="Method"))

,DS,PS,C-FID,MDD,ACD,SD,KD,ED,DTW,Time
Dataset,,,,,,,,,,
D2,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TransFusion,TimeVQVAE,TimeVQVAE,TransFusion,TransFusion,Time-Transformer
D3,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TimeVQVAE,TimeVQVAE,Time-Transformer
D4,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TimeVQVAE,TimeVQVAE,Time-Transformer,Time-Transformer,Time-Transformer
D5,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,Time-Transformer,Time-Transformer,Time-Transformer
D6,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TransFusion,TimeVQVAE,TimeVQVAE,Time-Transformer
D7,TransFusion,TimeGAN,TransFusion,TimeGAN,TransFusion,TimeVQVAE,TimeVQVAE,TransFusion,TransFusion,Time-Transformer


In [120]:
leaderboard(df_all[df_all.index.get_level_values(level=0).isin(["TTS-GAN", "TimeGAN"])])

,DS,PS,C-FID,MDD,ACD,SD,KD,ED,DTW,Time
Dataset,,,,,,,,,,
D2,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN
D3,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TTS-GAN
D4,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TTS-GAN
D5,TimeGAN,TTS-GAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN
D6,TTS-GAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TTS-GAN
D7,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TTS-GAN


In [121]:
leaderboard(df_all[df_all.index.get_level_values(level=0).isin(["Time-Transformer", "TimeGAN"])])

,DS,PS,C-FID,MDD,ACD,SD,KD,ED,DTW,Time
Dataset,,,,,,,,,,
D2,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,TimeGAN,Time-Transformer
D3,TimeGAN,TimeGAN,TimeGAN,Time-Transformer,TimeGAN,TimeGAN,TimeGAN,Time-Transformer,Time-Transformer,Time-Transformer
D4,TimeGAN,Time-Transformer,Time-Transformer,Time-Transformer,TimeGAN,TimeGAN,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer
D5,TimeGAN,TimeGAN,TimeGAN,TimeGAN,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer
D6,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer,Time-Transformer,TimeGAN,Time-Transformer,Time-Transformer,Time-Transformer
D7,TimeGAN,TimeGAN,TimeGAN,TimeGAN,Time-Transformer,TimeGAN,Time-Transformer,TimeGAN,TimeGAN,Time-Transformer


In [122]:
df = rankings
utility_cols = [col for col in df.columns if col[0] in ["PS"]]

fidelity_cols = [col for col in df.columns if col[0] not in ["PS", "Time"]]

timing_cols = [col for col in df.columns if col[0] in ["Time"]]

# Step 3: Compute averages
avg_utility = df[utility_cols].mean(axis=1)
avg_fidelity = df[fidelity_cols].mean(axis=1)
avg_timing = df[timing_cols].mean(axis=1)

# Step 4: Create new dataframe
new_df = pd.DataFrame({
    "Utility": avg_utility,
    "Fidelity": avg_fidelity,
    "Time": avg_timing
})

new_df.style.background_gradient(cmap=cm)

,Utility,Fidelity,Time
Method,,,
JustCopy,1.250000,1.000000,1.000000
TTS-GAN,5.166667,5.572917,5.000000
Time-Transformer,4.000000,3.645833,2.000000
TimeGAN,3.166667,3.541667,5.500000
TimeVQVAE,4.000000,3.135417,3.166667
TransFusion,1.833333,2.437500,4.333333


In [123]:
timings.set_index("Method")

,D2,D3,D4,D5,D6,D7
Method,,,,,,
JustCopy,0,0,0,0,0,0
TimeGAN,259,1373,1195,304,1252,1385
Time-Transformer,13,16,28,66,77,55
TransFusion,43,222,454,201,1387,856
TTS-GAN,646,660,649,649,660,645
TimeVQVAE,73,94,94,74,88,86
